# NFL Historic to Present Matches - Processing

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
out_df = pd.read_csv("Data/historic_match_scores.csv")
out_df["Date"] = out_df["Date"].astype("datetime64") 
out_df["Week"] = [row.lstrip().rstrip() for row in out_df["Week"].astype("str")]
out_df = out_df.drop(["Unnamed: 0", "Unnamed: 4", "Unnamed: 8"], axis=1)

* PtsW - Points scored by the winning team
* PtsL - Points score by the losing team
* YdsW - Yards gained by the winning team
* YdsL - Yards gained by the losing team
* TOW - Turnovers by winning Team
* TOL - Turnovers by losing Team

* Wild Card - Added in 1978 (March), being the 1977 season

In [3]:
def get_home_team(Winner, Loser, AtHome):
    if AtHome == "@":
        HomeTeam = Loser
    else:
        HomeTeam = Winner
    
    return HomeTeam

In [4]:
out_df["HomeTeam"] = out_df.apply(lambda row : get_home_team(row["Winner/tie"],
                                  row["Loser/tie"], row["AtHome"]), axis = 1)

In [5]:
def get_season(DateField):
    """This function parses the given match date to process and the return the year of the season being played.

    Args:
        DateField (datetime): The column/field that contains the date that the match was played upon

    Returns:
        Int: The year of the season being played
    """
    if (DateField.month >= 1) & (DateField.month <= 3):
        SeasonYear = DateField.year -1
    else:
        SeasonYear = DateField.year
    return SeasonYear

In [6]:
out_df["Season"] = out_df.apply(lambda row : get_season(row["Date"]), axis = 1)

In [7]:
out_df["Week"] = out_df["Week"].replace("Conf. Champ", "Championship Game")

In [8]:
out_df["PtsMargin"] = out_df["PtsW"] - out_df["PtsL"]
out_df["TieGame"] = np.where(out_df["PtsMargin"] == 0, 1, 0)

### Week Types

In [9]:
unique_weeks = {}
for season in out_df["Season"].unique():
    unique_weeks[season] = {}
    temp_df = out_df[out_df["Season"] == season]
    weeks_found = temp_df["Week"].unique().tolist()
    unique_weeks[season] = [str(i) for i in weeks_found]

In [10]:
all_time_unique = sorted({x for v in unique_weeks.values() for x in v})

### Max Week

* We want to obtain the highest week number present in each season
* Following this we can convert the post-season games to a week number, taking account for the fact that the number of weeks varies by season

In [11]:
max_week = {}
for season in out_df["Season"].unique():
    max_week[season] = {}
    temp_df = out_df["Week"][out_df["Season"] == season].astype("str")
    week_values = [int(i) for i in temp_df if not re.search('[a-zA-Z]', i)]
    #week_values = [int(i) for i in week_values]
    max_week[season] = max(week_values)

### Replacement Keys

* System by which we can replace the post-season week names with a numeric value, following on from the final regular game week for the season in question

In [12]:
week_correction_dict = {}
for season in max_week.keys():
    week_correction_dict[season] = {}
    temp_unique_weeks = unique_weeks[season] # how many numeric weeks (i.e. reg season)
    temp_max_weeks = max_week[season] # max week for that season
    
    if (season < 1966):
        week_correction_dict[season]["Division"] =temp_max_weeks + 1
        week_correction_dict[season]["Championship Game"] = temp_max_weeks + 2
    elif (season >= 1966) & (season <= 1978):
        week_correction_dict[season]["Division"] =temp_max_weeks + 1
        week_correction_dict[season]["Championship Game"] = temp_max_weeks + 2
        week_correction_dict[season]["Super Bowl"] = temp_max_weeks + 3
    else:
        week_correction_dict[season]["Wild Card"] = temp_max_weeks + 1
        week_correction_dict[season]["Division"] =temp_max_weeks + 2
        week_correction_dict[season]["Championship Game"] = temp_max_weeks + 3
        week_correction_dict[season]["Super Bowl"] = temp_max_weeks + 4

In [13]:
def get_modified_week_col(season_value, week_value):
    for old, new in week_correction_dict[season_value].items():
        new_week = week_value.replace(old, str(new))
    return new_week

In [14]:
out_df["ModifiedWeek"] = out_df.apply(lambda row : get_modified_week_col(row["Season"], row["Week"]), axis=1)

In [15]:
out_df.head()

,Rk,Week,Day,Date,Winner/tie,AtHome,Loser/tie,PtsW,PtsL,YdsW,TOW,YdsL,TOL,HomeTeam,Season,PtsMargin,TieGame,ModifiedWeek
0,1,5,Sun,1944-10-15,Cleveland Rams,@,Detroit Lions,20,17,177.0,3.0,237.0,6.0,Detroit Lions,1944,3,0,5
1,2,8,Sun,1947-11-09,Chicago Bears,NaN,Green Bay Packers,20,17,320.0,9.0,299.0,4.0,Chicago Bears,1947,3,0,8
2,3,15,Sun,1947-12-07,New York Yankees,@,Brooklyn Dodgers,20,17,358.0,3.0,186.0,1.0,Brooklyn Dodgers,1947,3,0,15
3,4,7,Sun,1951-11-11,Cleveland Browns,NaN,Philadelphia Eagles,20,17,204.0,3.0,310.0,4.0,Cleveland Browns,1951,3,0,7
4,5,11,Sun,1951-12-09,Washington Redskins,@,Chicago Cardinals,20,17,334.0,2.0,334.0,2.0,Chicago Cardinals,1951,3,0,11


In [16]:
nfl_tie_count = len(out_df[out_df['TieGame'] == 1])
nfl_tie_percentage = nfl_tie_count / len(out_df) * 100
print(f"Number of ties in NFL History: {nfl_tie_count} ({nfl_tie_percentage:.3f}%)")

Number of ties in NFL History: 316 (1.865%)


## Output the Processed File to CSV

In [17]:
out_df.to_csv("Data/processed_game_history.csv")